In [1]:
import numpy as np
import pandas as pd

In [2]:
def merge2cats(cat1, cat2, merge_dist=400/3600, names=['1', '2'], prms={}): 
    from astropy.coordinates import SkyCoord
    from astropy import units as u
    
    sc1 = SkyCoord(ra=np.array(cat1['RA']) * u.degree, dec=np.array(cat1['DEC']) * u.degree, frame='icrs')
    sc2 = SkyCoord(ra=np.array(cat2['RA']) * u.degree, dec=np.array(cat2['DEC']) * u.degree, frame='icrs')
    
    idx, d2d, _ = sc1.match_to_catalog_sky(sc2)
    matched = d2d.degree <= merge_dist
    print(len(cat1), 'obj in ', names[0])
    print(len(cat2), 'obj in ', names[1])
    print(np.count_nonzero(matched), 'matched')
    print(len(cat1) - np.count_nonzero(matched), 'unique from', names[0])
   
    cats = [cat1, cat2]
    for this_cat, that_cat, this_name, that_name, this_idx in zip(cats, cats[::-1], names, names[::-1],
                                                            [cat1.index[matched], idx[matched]]):
        this_cat[this_name] = True
        this_cat[that_name] = False
        this_cat[that_name].iloc[this_idx] = True
    
    for coord in ['RA', 'DEC']:
        cat1[names[1]+coord] = np.nan
        cat2[names[1]+coord] = cat2[coord]
        cat1[names[1]+coord].iloc[matched] = np.array(cat2[coord][idx[matched]])
    
    for prm in prms:
        cat1[prm] = prms[prm]
        cat1[prm].iloc[matched] = np.array(cat2[prm][idx[matched]])
    
    print(len(cat2) - len(set(idx[matched])), 'unique from', names[1])
    cat2['matched'] = False
    cat2['matched'].iloc[idx[matched]] = True
    cat2 = cat2[cat2['matched'] == False]
    cat2.index = np.arange(len(cat2))
    print(len(cat2), 'unique from ', names[1])
    
    return pd.concat([cat1, cat2])

In [4]:
for mpp_val in np.arange(0.8, 0.9, 0.01):
    all_f = pd.read_csv('/home/rt2122/Data/detected_cats/full_pz_all_found34_thr0.1_step8_no_fn_mpp_er.csv')
    brcat = pd.read_csv('/home/rt2122/Data/SRGz/brcat_mpp_er.csv')
    all_f = all_f[all_f['mpp_er'] >= mpp_val]
    all_f.index = np.arange(len(all_f))
    brcat = brcat[brcat['mpp_er'] >= mpp_val]
    brcat.index = np.arange(len(brcat))
    brcat = brcat.rename(columns={'max_pred' : 's/n'})
    gen_cat = merge2cats(all_f, brcat, names=['SZcat(AL)', 'SZcat(y-map)'], prms={'s/n' : np.nan})
    gen_cat.to_csv('/home/rt2122/Data/gen_cats_er/er_gen_all_found_brcat_prec{:.2f}.csv'.format(mpp_val), index=False)

1732 obj in  SZcat(AL)
234 obj in  SZcat(y-map)
164 matched
1568 unique from SZcat(AL)
70 unique from SZcat(y-map)
70 unique from  SZcat(y-map)
1665 obj in  SZcat(AL)
147 obj in  SZcat(y-map)
96 matched
1569 unique from SZcat(AL)
51 unique from SZcat(y-map)
51 unique from  SZcat(y-map)
1613 obj in  SZcat(AL)
112 obj in  SZcat(y-map)
66 matched
1547 unique from SZcat(AL)
46 unique from SZcat(y-map)
46 unique from  SZcat(y-map)
1544 obj in  SZcat(AL)
77 obj in  SZcat(y-map)
44 matched
1500 unique from SZcat(AL)
33 unique from SZcat(y-map)
33 unique from  SZcat(y-map)
1483 obj in  SZcat(AL)
45 obj in  SZcat(y-map)
29 matched
1454 unique from SZcat(AL)
16 unique from SZcat(y-map)
16 unique from  SZcat(y-map)
1419 obj in  SZcat(AL)
37 obj in  SZcat(y-map)
22 matched
1397 unique from SZcat(AL)
15 unique from SZcat(y-map)
15 unique from  SZcat(y-map)
1367 obj in  SZcat(AL)
28 obj in  SZcat(y-map)
15 matched
1352 unique from SZcat(AL)
13 unique from SZcat(y-map)
13 unique from  SZcat(y-map)
13